In [1]:
!pip install -r requirements.txt

  Using cached protobuf-3.19.6-cp39-cp39-win_amd64.whl (895 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.1
    Uninstalling protobuf-4.25.1:
      Successfully uninstalled protobuf-4.25.1


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.60.0 requires protobuf>=4.21.6, but you have protobuf 3.19.6 which is incompatible.


In [2]:
!pip install google-generativeai

  Using cached protobuf-4.25.1-cp39-cp39-win_amd64.whl (413 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.19.6
    Uninstalling protobuf-3.19.6:
      Successfully uninstalled protobuf-3.19.6


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit 1.22.0 requires protobuf<4,>=3.12, but you have protobuf 4.25.1 which is incompatible.
mysql-connector-python 8.0.32 requires protobuf<=3.20.3,>=3.11.0, but you have protobuf 4.25.1 which is incompatible.


In [3]:
!pip install InstructorEmbedding

In [5]:
!pip install torch

     ------------------------------------- 192.2/192.2 MB 11.3 MB/s eta 0:00:00


In [2]:
!pip install sentence-transformers

  Using cached sentence-transformers-2.2.2.tar.gz (85 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 8.2/8.2 MB 6.5 MB/s eta 0:00:00
     ---------------------------------------- 1.1/1.1 MB 14.5 MB/s eta 0:00:00
     ------------------------------------- 977.6/977.6 kB 15.4 MB/s eta 0:00:00
     ------------------------------------- 311.7/311.7 kB 18.8 MB/s eta 0:00:00
     -------------------------------------- 168.9/168.9 kB 9.9 MB/s eta 0:00:00
     ---------------------------------------- 2.2/2.2 MB 17.5 MB/s eta 0:00:00
     ------------------------------------- 277.8/277.8 kB 16.7 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125925 sha256=6147396b30e97e615edba1c40f6fd21d3d1c3d8796ed1768e42a04ad188e322d
  Stored in directory: c:\users\nicho\appdata\local\pip\cache\wheels\71\67\06\162a3760c40d74dd40bc855d5

In [6]:
from langchain.llms import GooglePalm
api_key = ""
llm = GooglePalm(google_api_key=api_key, temperature=0.7) # higher temperature equals more creative, lower temperature equals less creative

In [7]:
from langchain.document_loaders.csv_loader import CSVLoader
loader = CSVLoader(file_path='uwi_dcit_faqs.csv', source_column='prompt')
data = loader.load()

In [8]:
data

[Document(page_content='prompt: How many undergraduate programmes do you offer ?\nresponse: We offer 8 undergraduate courses.', metadata={'source': 'How many undergraduate programmes do you offer ?', 'row': 0}),
 Document(page_content='prompt: What undergraduate programmes do you offer ?\nresponse: At the undergraduate level we offer the following B.Sc. Programs for new students as of 2023/2024 semester: B.Sc. Computer Science (Special); B.Sc. Computer Science and Management; Major in Computer Science; Minor in Computer Science; B.Sc. Information Technology (Special); Major in Information Technology; Minor in Information Technology; B.Sc. Software Engineering (Mobile Applications Technologies.', metadata={'source': 'What undergraduate programmes do you offer ?', 'row': 1}),
 Document(page_content='prompt: What is the B.Sc. Computer Science (Special) programme / Degree ?\nresponse: The B.Sc. Computer Science (Special) programme consists of 3 levels of computer science and information te

## Create an embedding using hugging face instructor embeddings

[Link to more info on embeddings](https://python.langchain.com/docs/integrations/text_embedding/huggingfacehub)

In [9]:
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.vectorstores import FAISS

instruct_embeddings = HuggingFaceInstructEmbeddings()
# Using vectordb in memory for experimenting
vector_database = FAISS.from_documents(documents=data, embedding=instruct_embeddings)

load INSTRUCTOR_Transformer
max_seq_length  512


In [10]:
# This object job is whenever a question is asked it creates an embedding of that question then pull similar looking vectors from the vector db
retriever = vector_database.as_retriever(score_threshold=0.7)

In [12]:
retrieved_docs = retriever.get_relevant_documents("How about job placement support ?")
retrieved_docs

[Document(page_content='prompt: Will this programme guarantee me a job?\nresponse: The UWI provide many opportunities to prepare students for the World of Work. DCIT offers internships to gain real world experience while completing course credits and being paid. However, we want to be honest and do not want to make any impractical promises! Our guarantee is to prepare you for the job market by teaching the most relevant skills, knowledge & timeless principles good enough to fetch the job.', metadata={'source': 'Will this programme guarantee me a job?', 'row': 26}),
 Document(page_content='prompt: Do you provide any internships ?\nresponse: The department offers an optional internship programme for students pursuing a degree in Computer Science or Information Technology. Students participate in the programme by registering for either INFO 3609, a 3-credit or INFO 3610, a 6-credit Internship course. However, students are not allowed to read both INFO 3609 and INFO 3610. More details are 

## Resources

[Langchain repository hub](https://smith.langchain.com/hub)

[Python langchain documentation](https://python.langchain.com/docs/modules/data_connection/retrievers/web_research)

In [15]:
from langchain.prompts import PromptTemplate

prompt_template = """Given the following context and a question, generate an answer based on this context only.
In the answer try to provide as much text as possible from "response" section in the source document context without making much changes.
If the answer is not found in the context, kindly state "I don't know." Don't try to make up an answer.

CONTEXT: {context}

QUESTION: {question}"""


PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)
chain_type_kwargs = {"prompt": PROMPT}
from langchain.chains import RetrievalQA
retrieval_qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    input_key="query",
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs
)

In [16]:
retrieval_qa_chain("Do you provide job assistance and also do you provide job gurantee?")

{'query': 'Do you provide job assistance and also do you provide job gurantee?',
 'result': 'The UWI provide many opportunities to prepare students for the World of Work. DCIT offers internships to gain real world experience while completing course credits and being paid. However, we want to be honest and do not want to make any impractical promises! Our guarantee is to prepare you for the job market by teaching the most relevant skills, knowledge & timeless principles good enough to fetch the job.',
 'source_documents': [Document(page_content='prompt: Will this bootcamp guarantee me a job?\nresponse: Our guarantee is to prepare you for a career in tech and provide you with an introduction to the topics covered at the university.', metadata={'source': 'Will this bootcamp guarantee me a job?', 'row': 14}),
  Document(page_content='prompt: Will this programme guarantee me a job?\nresponse: The UWI provide many opportunities to prepare students for the World of Work. DCIT offers internshi